<a href="https://colab.research.google.com/github/BecomeAllan/S2Search/blob/main/SemanticScholarSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Consumindo a API do SemanticScholar

A seguir, tem uma classe chamada `Search()`, que ao instanciar-la em uma variável é possível fazer pesquisas sobre papers utilizando a api do SemanticScholar, dentre os parâmetros temos:

- Buscar: Pesquisas sobre tópicos onde adicionar tópicos utiliza-se + (mais) e remover tópicos usamos - (menos)

  ex. "Machine+Medicine"

- Fields: O que será retornado como dados. Para utilizar, escolha dentre as opções sem utilizar espaço e separadas de virgulas:
  - (str): externalIds
  - (str): url
  - (str): title
  - (str): abstract
  - (str): venue 
  - (str): year 
  - (str): referenceCount
  - (str): citationCount
  - (str): influentialCitationCount
  - (str): isOpenAccess
  - list (str): fieldsOfStudy
  - list (str): authors 

  ex. "title,abstract,isOpenAccess,fieldsOfStudy"

- Offset: Número que começa a puxar a partir da ordem dele a lista de papers. (0 seria o primeiro)

- Limite: Número de papers a ser retornados (Máx. 10.000)

**Obs:** A api do SemanticScholar disponibiliza 100 query's a cada 5 min, no qual apenas retorna no máx. 100 resutados (limite). Assim a cada 5 min, é possível puxar 10.000 papers.

In [12]:
#@title Classe para pesquisa no SemanticScholar
import IPython
from google.colab import output
import pandas as pd

class Search():
  def __init__(self, **kwargs):
    self.data = ""
    self.data_0 = ""

    self.search = kwargs.get('search', None)
    self.fields = kwargs.get('fields', None)
    self.limit = kwargs.get('limit', None)
    self.offset = kwargs.get('offset', None)

    if self.search == None and self.fields == None and self.limit == None and self.offset == None:
      self._start(False)
    else:
      self._start(True)
  
  def _start(self, *args):

    output.register_callback('notebook.searching', self._searching)
    output.register_callback('notebook.AddListItem', self._add_list_item)
    output.register_callback('notebook.mergeData', self._merge_data)
    output.register_callback('notebook.error', self._error)


    boxs = ''' 
        <label for="query">Buscar: </label>
        <input type="text" id="query" value="Machine Learning+Deep Learning" style="width: 400px;"/>
        <br/>
        <br/>
        
        <label for="fields">Fields: </label>
        <input type="text" id="fields" value="title,abstract,isOpenAccess,fieldsOfStudy" style="width: 400px;"/>
        <br/>
        <br/>
 
        <label for="limit">Limite: </label>
        <input type="text" id="limit" value="10" style="width: 50px;"/><br/>
        <br/>

        <label for="limit">Offset: </label>
        <input type="text" id="offset" value="0" style="width: 50px;"/><br/>
        <br/>

        <button id='button'>Pesquisar</button>
        <br/>
        <br/>
           '''

    button = ''' document.querySelector('#button').onclick = async () => ''' # {}

    search_query = '''
            var search = document.getElementById("query").value
            var fields = document.getElementById("fields").value
            var limit = parseInt(document.getElementById("limit").value)
            var offset = parseInt(document.getElementById("offset").value)
                  '''
    search_params = '''
            var search = "{search}"
            var fields = "{fields}"
            var limit = parseInt({limit})
            var offset = parseInt({offset})
                  '''
    engine = '''
            google.colab.kernel.invokeFunction('notebook.searching', [], {});

            if (limit >100) {
              var number = limit
              var data = ""
              var promises = []
              var offsetSearch = 0
              var rest = 0

              for (let index = 0; index < Math.floor(limit/100); index++) {
                offsetSearch = 100*(index) + offset + 1*(index!==0)


                promises.push(
                  fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=100&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )
              }
              
              if (limit%100 !== 0) { 
                rest= limit%100
                offsetSearch = offsetSearch+100
                
                console.log(rest)
                console.log(offsetSearch)

                promises.push(
                fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offsetSearch}&limit=${rest}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {return(res)})
                )}

              await Promise.all(promises).then(data=>{
                google.colab.kernel.invokeFunction('notebook.mergeData', [data], {})
              })
              .catch(err=> { return (google.colab.kernel.invokeFunction('notebook.error', [err], {})) })

            } else {

            await fetch(`https://api.semanticscholar.org/graph/v1/paper/search?query=${search}&offset=${offset}&limit=${limit}&fields=${fields}`)
    .then(res=> {return(res.json())})
    .then(res=> {
      console.log(res)
      console.log("AQUIII")
      return(google.colab.kernel.invokeFunction('notebook.AddListItem', [res], {}))})
    .catch(err=> { return (
      google.colab.kernel.invokeFunction('notebook.error', [err], {})) })
            }
                  '''

    asyncfun = "async function asyncfun()"

    if args[0]:

      main_app =  "<script>" + search_params.format(search=self.search, fields=self.fields, limit=self.limit, offset=self.offset) + asyncfun + "{" + engine + "}" + "asyncfun()" + "</script>"

      display(IPython.display.HTML(main_app))
      
    else:
      main_app = boxs + "<script>" + button + "{" + search_query + engine + "}" + "</script>"
      
      display(IPython.display.HTML(main_app))

    

  def _error(self,value):
    try:
      print("ERRO na API SemanticScholar:\n")
      print(value)
    except:
      pass 

  def _searching(self):
    with output.use_tags('some_outputs'):
      print("\n\nPesquisando...")
      sys.stdout.flush();

  def _merge_data(self, data):
    output.clear(output_tags='some_outputs')
    print(f"Achou {data[0]['total']} papers.\n")
    self.data_0 = data

    self.data = pd.DataFrame(data[0]['data'])

    try:
      for x in data[1:len(data)]:
        try:
          self.merge(pd.DataFrame(x['data']))
        except:
          self._error(x)
    except:
      pass 

    print(f"\nApi devolveu >> {self.data.shape[0]} papers\n" )
    print(self.data.head())


  def merge(self, data):
    self.data = pd.concat([self.data, data], ignore_index=True ) 

  def _add_list_item(self, value):
    output.clear(output_tags='some_outputs')

    print(f"Achou {value['total']} papers.\n")

    self.data = pd.DataFrame(value['data'])

    print(f"Api devolveu >> {self.data.shape[0]} papers\n" )
    
    print(self.data.head())



# Consumir a classe `Search()`

A duas formas de pesquisar utilizando `Search()`:

1. A primeira é utilizando parâmetros na propria classe:

In [ ]:
# "Decision making" AND "optimization" AND "artificial intelligence"

In [31]:
Resultados = Search(search = "Machine Learning+Deep Learning" , fields = "title,abstract,citationCount,isOpenAccess,fieldsOfStudy", limit = "2000", offset = "0")

Achou 5240039 papers.

ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}
ERRO na API SemanticScholar:

{'message': 'Internal Server Error'}

Api devolveu >> 1009 papers

                                    paperId  ...                    fieldsOfStudy
0  46200b99c40e8586c8a0f588488ab6414119fb28  ...               [Computer Science]
1  9c9d7247f8c51ec5a02b0d911d1d7b9e8160495d  ...               [Computer Science]
2  b42b1bfdc262bf99e9484e2e9df94df216b96374  ...               [Computer Science]
3  25badc676197a

In [30]:
# Os dados ficam na variável data, no qual é uma tabela do tipo pandas
print(Resultados.data.columns)
print(Resultados.data.sort_values("citationCount", ascending = False ).head())

,paperId,title,abstract,citationCount,isOpenAccess,fieldsOfStudy
754,34f25a8704614163c4095b3ee2fc969b60de4698,Dropout: a simple way to prevent neural networ...,Deep neural nets with a large number of parame...,24510,False,[Computer Science]
17,a4cec122a08216fe8a3bc19b22e78fbaea096256,Deep Learning,Machine-learning technology powers many aspect...,19531,False,"[Medicine, Computer Science]"
963,5d90f06bb70a0a3dced62413346235c02b1aa086,Learning Multiple Layers of Features from Tiny...,Groups at MIT and NYU have collected a dataset...,13232,False,[Computer Science]
988,6bdb186ec4726e00a8051119636d4df3b94043b5,Caffe: Convolutional Architecture for Fast Fea...,Caffe provides multimedia scientists and pract...,13101,False,[Computer Science]
14,46200b99c40e8586c8a0f588488ab6414119fb28,TensorFlow: A system for large-scale machine l...,TensorFlow is a machine learning system that o...,10598,False,[Computer Science]


2. A segunda é atravez da api de busca, searchBox, no qual é possivel colocar os campos:

In [21]:
Resultados_2 = Search()

Achou 652200 papers.

Api devolveu >> 10 papers

                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
3  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
4  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]

[5 rows x 5 columns]


In [22]:
print(Resultados_2.data)

                                    paperId  ...       fieldsOfStudy
0  846ff7afb7670d62f88b4a8cc99d306ffb81b075  ...          [Medicine]
1  5dc53e50148b01fe8b9536eb79fa6b1dce924174  ...          [Medicine]
2  7cc2e148d27a7508dd23c4e35eb63cc9b3e6a58f  ...  [Computer Science]
3  59444b096f7c8a561d540102e8b5bfb189edabc6  ...                None
4  eee313380ccb45807ea0afa3c1df86f6b48b8867  ...  [Computer Science]
5  46479bbea7749cb2db35b139206039531327053c  ...  [Computer Science]
6  b69fe5a837277ddbea5215d6bacd3a902e9d11ce  ...          [Medicine]
7  b0bf64ccbd651e8c7bc141d8aabaecff562e93a1  ...  [Computer Science]
8  042ab08ec6782cf217f13175162bfd48f7350114  ...  [Computer Science]
9  03e7832982986159400a8eeab148487ffcfabe56  ...  [Computer Science]

[10 rows x 5 columns]


# **ON BUILD...**

In [ ]:
#### NOT RUN

## Para fazer a requisição de dados, é utilizado o metodo POST para HTTP
import requests
import json

url = "https://www.semanticscholar.org/api/1/search"

PARAMS_FOR_POST = '''{
    "queryString": "Machine Learning+Deep Learning",
    "page": 1,
    "pageSize": 10,
    "sort": "total-citations", #influence #"pub-date" #relevance
    "authors": [],
    "coAuthors": [],
    "venues": [
        "PloS one",
        "AAAI",
        "Scientific reports",
        "IEEE Access",
        "ArXiv",
        "Expert Syst. Appl.",
        "ICML",
        "Neurocomputing",
        "Sensors",
        "Remote. Sens."
    ],
    "yearFilter": {
        "min": 2008,
        "max": 2021
    },
    "requireViewablePdf": true,
    "publicationTypes": [
        "ClinicalTrial",
        "CaseReport",
        "Editorial",
        "Study",
        "Book",
        "News",
        "Review",
        "Conference",
        "LettersAndComments",
        "JournalArticle"
    ],
    "externalContentTypes": [],
    "fieldsOfStudy": [
        "biology",
        "art",
        "business",
        "computer-science",
        "chemistry",
        "economics",
        "engineering",
        "environmental-science",
        "geography",
        "geology",
        "history",
        "materials-science",
        "mathematics",
        "medicine",
        "philosophy",
        "physics",
        "political-science",
        "psychology",
        "sociology"
    ],
    "useFallbackRankerService": false,
    "useFallbackSearchCluster": false,
    "hydrateWithDdb": true,
    "includeTldrs": true,
    "performTitleMatch": true,
    "includeBadges": true,
    "tldrModelVersion": "v2.0.0",
    "getQuerySuggestions": false
}
'''
# post = json.loads(data)
# res = requests.post(url, json=post)
# res.text

In [34]:
import requests

class SearchWeb():
  def __init__(self, search="Machine Learning+Deep Learning", **kwargs):
     
    self.search = search
    self.sort = kwargs.get('sort', "relevance")
    self.authors = kwargs.get('authors', [])
    self.coAuthors = kwargs.get('coAuthors', [])
    self.venues = kwargs.get('venues', ['PloS one', 'AAAI', 'Scientific reports', 'IEEE Access', 'ArXiv', 'Expert Syst. Appl.', 'ICML', 'Neurocomputing', 'Sensors', 'Remote. Sens.'])
    self.yearFilter = kwargs.get('yearFilter', None) # {"min": 2008,"max": 2021}
    self.requireViewablePdf = kwargs.get('requireViewablePdf', False)
    self.publicationTypes = kwargs.get('publicationTypes', ["ClinicalTrial", "CaseReport", "Editorial","Study","Book","News","Review","Conference","LettersAndComments","JournalArticle"])
    self.fieldsOfStudy = kwargs.get('fieldsOfStudy', ["biology","art","business","computer-science","chemistry","economics","engineering","environmental-science","geography","geology","history","materials-science","mathematics","medicine","philosophy","physics","political-science","psychology","sociology"])
    self.useFallbackRankerService = kwargs.get('useFallbackRankerService', False)
    self.useFallbackSearchCluster = kwargs.get('useFallbackSearchCluster', False)
    self.hydrateWithDdb = kwargs.get('hydrateWithDdb', True)
    self.includeTldrs = kwargs.get('includeTldrs', True)
    self.performTitleMatch = kwargs.get('performTitleMatch', True)
    self.includeBadges = kwargs.get('includeBadges', True)
    self.tldrModelVersion = kwargs.get('tldrModelVersion', 'v2.0.0')
    self.getQuerySuggestions = kwargs.get('getQuerySuggestions', False)


    self.post = {
    "queryString": self.search,
    "pageSize": 10,
    "sort": self.sort,
    "authors": self.authors,
    "coAuthors": self.coAuthors,
    "venues": self.venues,
    "yearFilter": self.yearFilter,
    "requireViewablePdf": self.requireViewablePdf,
    "publicationTypes": self.publicationTypes,
    "externalContentTypes": [],
    "fieldsOfStudy": self.fieldsOfStudy,
    "useFallbackRankerService": self.useFallbackRankerService,
    "useFallbackSearchCluster": self.useFallbackSearchCluster,
    "hydrateWithDdb": self.hydrateWithDdb,
    "includeTldrs": self.includeTldrs,
    "performTitleMatch": self.performTitleMatch,
    "includeBadges": self.includeBadges,
    "tldrModelVersion": "v2.0.0",
    "getQuerySuggestions": self.getQuerySuggestions,
    }

    print('.post >>')
    print(self.post)

  def _search(self):
    url = "https://www.semanticscholar.org/api/1/search"
    return requests.post(url, json=self.post)


  def get(self, n = 10):
    self.all = {"Results": []}
    print("Searching")
    print(self.all)


    if n > 10:
      for page in range(n//10):

        self.post["page"] = page+1

        res = self._search()

        try:
          res_json = json.loads(res.text)
          self.all["Results"].extend([{"Page": self.post["page"], "N_Papers":len(res_json["results"]), "Papers":res_json["results"]}])
          # self.all.extend(res_json["results"])
        except:
          print(res.text) 

      if n%10>0:
        self.post["page"] = self.post["page"]+1
        self.post["pageSize"] = n%10
        res = self._search()


        try:
          res_json = json.loads(res.text)
          self.all["Results"].extend([{"Page": self.post["page"], "N_Papers":len(res_json["results"]), "Papers":res_json["results"]}])
          # self.all.extend(res_json["results"])
        except:
          print(res.text) 

    else:
      self.post["page"] = 1
      self.post["pageSize"] = n

      res = self._search()

      try:
        res_json = json.loads(res.text)
        self.all["Results"].extend([{"Page": self.post["page"], "N_Papers":len(res_json["results"]), "Papers":res_json["results"]}])
        # self.all.extend(res_json["results"])
      except:
        print(res.text) 

    print(self.all)

In [35]:
from_Webpage = SearchWeb(search= "Machine Learning+Deep Learning", sort= "total-citations" )

.post >>
{'queryString': 'Machine Learning+Deep Learning', 'pageSize': 10, 'sort': 'total-citations', 'authors': [], 'coAuthors': [], 'venues': ['PloS one', 'AAAI', 'Scientific reports', 'IEEE Access', 'ArXiv', 'Expert Syst. Appl.', 'ICML', 'Neurocomputing', 'Sensors', 'Remote. Sens.'], 'yearFilter': None, 'requireViewablePdf': False, 'publicationTypes': ['ClinicalTrial', 'CaseReport', 'Editorial', 'Study', 'Book', 'News', 'Review', 'Conference', 'LettersAndComments', 'JournalArticle'], 'externalContentTypes': [], 'fieldsOfStudy': ['biology', 'art', 'business', 'computer-science', 'chemistry', 'economics', 'engineering', 'environmental-science', 'geography', 'geology', 'history', 'materials-science', 'mathematics', 'medicine', 'philosophy', 'physics', 'political-science', 'psychology', 'sociology'], 'useFallbackRankerService': False, 'useFallbackSearchCluster': False, 'hydrateWithDdb': True, 'includeTldrs': True, 'performTitleMatch': True, 'includeBadges': True, 'tldrModelVersion': 'v2

In [36]:
# Retorna 2 papers com base nos parametros passados em SearchWeb() que constitui ().post
from_Webpage.get(n = 2)

Searching
{'Results': []}
{'Results': [{'Page': 1, 'N_Papers': 2, 'Papers': [{'id': '9c9d7247f8c51ec5a02b0d911d1d7b9e8160495d', 'corpusId': 5707386, 'title': {'text': 'TensorFlow: Large-Scale Machine Learning on Heterogeneous Distributed Systems', 'fragments': [{'start': 24, 'end': 31}, {'start': 32, 'end': 40}]}, 'slug': 'TensorFlow:-Large-Scale-Machine-Learning-on-Systems-Abadi-Agarwal', 'paperAbstract': {'text': 'TensorFlow is an interface for expressing machine learning algorithms, and an implementation for executing such algorithms. A computation expressed using TensorFlow can be executed with little or no change on a wide variety of heterogeneous systems, ranging from mobile devices such as phones and tablets up to large-scale distributed systems of hundreds of machines and thousands of computational devices such as GPU cards. The system is flexible and can be used to express a wide variety of algorithms, including training and inference algorithms for deep neural network models,

In [37]:
# Tudo que vem com base em 1 paper (dict.)
from_Webpage.all["Results"][0]["Papers"][0]

{'alternatePaperLinks': [],
 'authors': [[{'bitmap$0': False,
    'ids': ['2057642721'],
    'name': 'Martín Abadi',
    'slug': 'Martín-Abadi'},
   {'fragments': [], 'text': 'Martín Abadi'}],
  [{'bitmap$0': False,
    'ids': ['2078528337'],
    'name': 'Ashish Agarwal',
    'slug': 'Ashish-Agarwal'},
   {'fragments': [], 'text': 'Ashish Agarwal'}],
  [{'bitmap$0': False,
    'ids': ['144758007'],
    'name': 'P. Barham',
    'slug': 'P.-Barham'},
   {'fragments': [], 'text': 'P. Barham'}],
  [{'bitmap$0': False,
    'ids': ['2445241'],
    'name': 'E. Brevdo',
    'slug': 'E.-Brevdo'},
   {'fragments': [], 'text': 'E. Brevdo'}],
  [{'bitmap$0': False,
    'ids': ['2545358'],
    'name': 'Z. Chen',
    'slug': 'Z.-Chen'},
   {'fragments': [], 'text': 'Z. Chen'}],
  [{'bitmap$0': False,
    'ids': ['48738717'],
    'name': 'C. Citro',
    'slug': 'C.-Citro'},
   {'fragments': [], 'text': 'C. Citro'}],
  [{'bitmap$0': False,
    'ids': ['32131713'],
    'name': 'G. Corrado',
    'slug':